In [20]:
meeting_id = 'd894ee56-d144-4cd5-8c2c-74de222a6b31'

In [8]:
meeting_id = '6cf0b989-5e9c-4a57-a058-8f6ac0ca8025'

In [21]:
from psql_helpers import get_meeting_by_id,async_session
import json
import pandas as pd

async with async_session() as session:
    meeting = await get_meeting_by_id(session, meeting_id)


In [22]:
df = pd.DataFrame(eval(meeting.transcript))
df.head()




,speaker,speaker_id,content,html_content,html_content_short,keywords,timestamp
0,TBD,TBD,Привет!,Привет!,,[],2024-10-03T09:00:51.040000Z
1,Dmitriy Grankin,TBD,"Да, Оля, привет! Как поживаешь?","Да, <b>Оля</b>, привет! Как поживаешь?",Оля,[],2024-10-03T09:00:53.340000Z
2,Olga Mykhoparkina,TBD,"Да ничего, потихоньку, что-то приболела чуть-чуть.","Да ничего, потихоньку. Чуть-чуть приболела.",,[],2024-10-03T09:00:57.136000Z
3,Olga Mykhoparkina,TBD,Как ваши дела?,Как <b>ваши дела</b>?,ваши дела,[],2024-10-03T09:00:58.236000Z
4,Olga Mykhoparkina,TBD,Да ничего.,Да ничего.,,[],2024-10-03T09:00:59.496000Z


In [23]:
# Create a mask for speaker changes
speaker_change = (df['speaker'] != df['speaker'].shift())

# Create groups that reset when speaker changes or when same speaker speaks more than 3 times
group_id = (speaker_change | (df.groupby((speaker_change).cumsum())['speaker'].transform('size') > 5)).cumsum()

# Group the data and display results
grouped_df = df.groupby(group_id)

In [24]:
# Aggregate the groups by joining the text content
grouped_text = grouped_df.agg({
    'speaker': 'first',  # take first speaker name in group
    'content': ' '.join,    # join all text content
    'timestamp': 'first' # take first timestamp
})

In [25]:
grouped_df

In [26]:
from search import SearchAssistant
sa = SearchAssistant()
await sa.initialize()

In [27]:
grouped_text.head(10)

,speaker,content,timestamp
speaker,,,
1,TBD,Привет!,2024-10-03T09:00:51.040000Z
2,Dmitriy Grankin,"Да, Оля, привет! Как поживаешь?",2024-10-03T09:00:53.340000Z
3,Olga Mykhoparkina,"Да ничего, потихоньку, что-то приболела чуть-чуть. Как ваши дела? Да ничего.",2024-10-03T09:00:57.136000Z
4,Dmitriy Grankin,"Как там у Жени? Да, сейчас вот пошла, спортом занимается, на падл играет. А каким спортом?",2024-10-03T09:01:05.036000Z
5,Olga Mykhoparkina,Падл.,2024-10-03T09:01:14.286000Z
6,TBD,"Ух ты, прикольно.",2024-10-03T09:01:16.186000Z
7,Olga Mykhoparkina,"Не слышалось. Ну, вы как переехали, так и всё. На танцы не ходят. А, ну да,",2024-10-03T09:01:19.184000Z
8,Dmitriy Grankin,"видишь, далеко теперь. Ну да, да, далеко ездить.",2024-10-03T09:01:27.404000Z
9,Olga Mykhoparkina,"Да. Мы там в этом году собираемся выступать, там Женя, ну Женя, который тренер, там организовывает, там нашла какой-то конкурс в Лиссабоне. Будем там готовить formation, короче, там большие планы. Ну, класс.",2024-10-03T09:01:30.624000Z


In [28]:
# Import required libraries
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load a pre-trained sentence transformer model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Generate embeddings for all content
embeddings = model.encode(grouped_text['content'].values)

# Function to find similar sentences
def find_similar_sentences(query, embeddings, texts, top_k=20):
    # Encode the query
    query_embedding = model.encode([query])
    
    # Calculate similarities
    similarities = cosine_similarity(query_embedding, embeddings)[0]
    
    # Get top k similar indices
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    
    # Return results with similarities
    results = []
    for idx in top_indices:
        results.append({
            'text': texts.iloc[idx],
            'similarity': similarities[idx],
            'speaker': grouped_text.iloc[idx]['speaker'],
            'timestamp': grouped_text.iloc[idx]['timestamp']
        })
    
    return results

# Example usage:
# similar_segments = find_similar_sentences("your search query here", embeddings, grouped_text['content'])

In [29]:
pd.options.display.max_colwidth = 1000

In [30]:
q = "Surfer SEO"

In [31]:
r= find_similar_sentences(q, embeddings, grouped_text['content'])
pd.DataFrame(r)

,text,similarity,speaker,timestamp
0,"Ну, смотри, я рассказывала, что Surfer SEO, в принципе, делает что-то похожее, вот, они там, не помню, то ли 8 лет на рынке, короче, уже это делают. У них есть контент-эдитор. Ну, я не знаю, сейчас могу тебя открыть и расшарить, чтобы ты понимал, как это выглядит.",0.791058,Olga Mykhoparkina,2024-10-03T09:04:26.424000Z
1,"Тут еще у Surfer SEO есть такая фича, появилась, называется Humanizer, типа Content Humanizer.",0.708441,Olga Mykhoparkina,2024-10-03T09:16:33.644000Z
2,"В Surfer тоже есть такая фича, чтобы, допустим, там добавить...",0.699107,Olga Mykhoparkina,2024-10-03T09:18:19.620000Z
3,"Контент-эдитер, вот это мы сейчас в интерфейсе Surfer.co, у них есть вот такая штука, это контент-эдитер, то есть по сути, что он делает, это, ну, вот это все наши статьи, которые мы для клиентов там...",0.634054,Olga Mykhoparkina,2024-10-03T09:05:54.442000Z
4,"Мне просто интересно, знаешь, что сделать? Я бы взяла, допустим, какой-то ключ, который условно нас интересует, и сгенерила бы статью, Ну, и дальше бы проверила ее с помощью Surfer, вот этого эдитора, и посмотреть, какой, допустим, скор, да, даст Surfer. Ну, так, ради интереса. И как бы на основании этого уже дальше.",0.597158,Olga Mykhoparkina,2024-10-03T09:36:30.670000Z
5,Он SEO-шник в маркетинговом агентстве большом.,0.434245,Dmitriy Grankin,2024-10-03T09:03:29.398000Z
6,"Ну, это мы больше говорим про, я не знаю, overall техническую SEO-шку, там, условно, скорость загрузки страницы, там, да, вот эти все вещи, Всякие ошибки на сайте, там, в целом, там, внутренняя перелинковка, не знаю, какие-то там, ну, дубликатов, там, чтобы не было, в принципе, правильная структура.",0.423531,Olga Mykhoparkina,2024-10-03T09:39:59.420000Z
7,"Это очень важно для SEO, чтобы эти internal links были.",0.420407,Olga Mykhoparkina,2024-10-03T09:18:49.112000Z
8,"Да, интересно. А скажи мне, пожалуйста, давай тогда сделаем по SEO-шке следующим образом. Ну, я бы хотела сделать тестовую, вообще посмотреть, типа, что будет генерить. какую-то, не знаю, там, ну, даже одну какую-то, вот. И если, типа, it's okay, ну, то есть если я посмотрю, что я могу этот контент, условно, там, небольшими усилиями доработать и запустить, На блог себе запостить, вот, тогда, в принципе, можно уже там рассматривать дальше там какие-то там десятки статей там и там что-то там для твоего сайта. Авария перед серфингом.",0.381839,Olga Mykhoparkina,2024-10-03T09:34:38.484000Z
9,"Суть по сути та же, да, то есть чем больше классных ресурсов на тебя ссылается на твоей странице сайта, тем у тебя домен рейтинга выше.",0.379655,Olga Mykhoparkina,2024-10-03T09:19:59.326000Z


In [62]:

global_results = await sa.search(q, user_id=va.user_id)
global_results.head(20)



/home/dima/playground/dev_multiassistant/back/search.py:114: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([main_df, speaker_df]).drop_duplicates(subset=columns).reset_index(drop=True)


,topic_name,speaker_name,summary,details,meeting_id,timestamp,score,vector_scores,exact_matches,source
0,Surfer SEO,Olga Mykhoparkina,Surfer SEO is a tool used for optimizing content based on SEO metrics.,"Olga mentions Surfer SEO's content editor, which helps in analyzing and improving the quality of articles based on competitor analysis.",d894ee56-d144-4cd5-8c2c-74de222a6b31,2024-10-03T09:00:51.040000,1.500000,"{'topic_name': 0.7000000500000001, 'summary': 0.59580548, 'details': 0.46483810000000003}","[topic_name, summary, details, topic_name, summary, details, topic_name, summary, details]",main
1,Surfer SEO,Rick Tousseyn,Surfer SEO is a tool that helps users optimize their content based on data-driven insights.,Rick discusses Surfer SEO in the context of creating content clusters and improving SEO strategies.,6cf0b989-5e9c-4a57-a058-8f6ac0ca8025,2024-10-03T18:06:40.070000,1.500000,"{'topic_name': 0.75000005, 'summary': 0.636142225, 'details': 0.5898206500000001}","[topic_name, summary, details, topic_name, summary, details, topic_name, summary, details]",main
2,Surfer SEO Utilization,Olga Mykhoparkina,Olga introduced Surfer SEO as a tool for optimizing content and improving article rankings on search engines.,"Surfer SEO provides a content editor that analyzes competitors and suggests improvements based on keyword usage, article length, and other factors. Olga suggested using it to enhance the quality of articles for better SEO performance.",d894ee56-d144-4cd5-8c2c-74de222a6b31,2024-10-03T09:00:51.040000,1.448214,"{'topic_name': 0.7241072, 'details': 0.59890422, 'summary': 0.560236425}","[topic_name, details, summary, topic_name, details, summary, topic_name, details, summary]",main
3,Surfer,Rick Tousseyn,Surfer is an SEO tool mentioned by Rick that helps optimize content based on keyword analysis and competition.,"Rick uses Surfer to benchmark content and improve SEO strategies, highlighting its features and benefits.",bdc05628-4278-4cda-9395-49906c348d7d,2024-10-01T09:19:05.583999,0.919880,"{'topic_name': 0.509939825, 'summary': 0.488020425, 'details': 0.463090875}",[],main
4,SEO,Rick Tousseyn,"SEO (Search Engine Optimization) is a key focus of the meeting, with discussions on tools and strategies to improve content ranking.","Rick, as the head of SEO at Team Lewis, shares insights on SEO tools and their importance in content optimization.",bdc05628-4278-4cda-9395-49906c348d7d,2024-10-01T09:19:05.583999,0.724903,{'topic_name': 0.7249034999999999},[],main
5,SEO,Rick Tousseyn,SEO (Search Engine Optimization) is the practice of optimizing websites to rank higher in search engine results.,The discussion revolves around using plugins like Rank Math and Yoast to improve SEO for Dmitriy's WordPress site.,6cf0b989-5e9c-4a57-a058-8f6ac0ca8025,2024-10-03T18:06:40.070000,0.724903,"{'topic_name': 0.7249034999999999, 'summary': 0.351805118}",[],main
6,SEO,Olga Mykhoparkina,"SEO stands for Search Engine Optimization, a strategy to improve website visibility in search engines.","The meeting covers various aspects of SEO, including content optimization, keyword usage, and the importance of internal linking.",d894ee56-d144-4cd5-8c2c-74de222a6b31,2024-10-03T09:00:51.040000,0.724903,"{'topic_name': 0.7249034999999999, 'summary': 0.38374009800000003}",[],main
7,SEO,Сергей,SEO (Search Engine Optimization) is a strategy discussed by Dmitry and Сергей to improve the visibility of VEX.AI's product in search engines.,They discuss the importance of optimizing the product's online presence to attract more users and improve search rankings.,0aee5d35-5e04-4b48-b428-6e2979aab55c,2024-09-25T13:28:55.560000,0.724903,{'topic_name': 0.7249034999999999},[],main
8,SEO,Dmitry Grankin,SEO is mentioned as a strategy to improve visibility and attract users.,The speakers discuss using SEO to extract specific cases and improve the product's online presence.,e2f3a86b-e94d-4967-8dad-89ac78a0617c,2024-09-13T10:17:35.355999,0.724903,"{'topic_name': 0.724903499999999

In [28]:
from vexa import VexaAPI
va = VexaAPI()
await va.get_user_info()
va.user_id



Vexa token: 3ae04e20124d40babc5107e658c666b6
User information retrieved successfully.


'ef7c085b-fdb5-4c94-b7b6-a61a3d04c210'